# Position Reconstruction for XeBRA - Measurements Analysis

**Status:** August 2019

Position reconstruction for XeBRA following the position reconstruction algorithm of Mercury (employed in the LUX experiment, originally developed for the ZEPLIN-III dark matter experiment) with light response functions (see https://arxiv.org/abs/1710.02752v2 and https://arxiv.org/abs/1112.1481 ).

In [1]:
#### Imports:

import sys

import numpy as np
from numpy import exp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
from matplotlib.image import NonUniformImage
from matplotlib.patches import Rectangle
%matplotlib inline

import pandas as pd
import math

from scipy import optimize
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy import stats
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.interpolate import spline
from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import griddata
from scipy.stats import chisquare
from scipy.stats import power_divergence

## Imports and Data Processing

In [2]:
import uproot # TODO: remove

## General Definitions and Parameters

In [3]:
#### General function definitions:

def f_polar_R(cartesian_x, cartesian_y):
    return np.sqrt(cartesian_x * cartesian_x + cartesian_y * cartesian_y)

def f_polar_Phi(cartesian_x, cartesian_y):
    return np.arctan2(cartesian_y, cartesian_x)

def f_distance(x_a, y_a, x_b, y_b):
    return np.sqrt((x_b - x_a)**2 + (y_b - y_a)**2)

In [4]:
#### Fixed input parameters

## PMT positions
PMT_ID = np.array([1,2,3,4,5,6,7])
PMT_position_x = np.array([-14.,-28,-14.,14.,28.,14.,0.])      # x-position PMTs in mm in cartesian coordinates
PMT_position_y = np.array([-28.,0.,28.,28.,0.,-28.,0.])        # y-position PMTs in mm in cartesian coordinates
PMT_position_R = f_polar_R(PMT_position_x, PMT_position_y)     # radial position PMTs in mm in polar coordinates
PMT_position_Phi = f_polar_Phi(PMT_position_x, PMT_position_y) # angular position PMTs in rad in polar coordinates
PMT_positions = pd.DataFrame(index=PMT_ID, data={'PMT_ID': PMT_ID, 'PMT_position_x': PMT_position_x, 'PMT_position_y': PMT_position_y, 'PMT_position_R': PMT_position_R, 'PMT_position_Phi': PMT_position_Phi})

## Radius S2 region used to constrain radius reconstructed position
p_TPC_radius = 35

## Fit parameters LRFs
## MC driven (R_PTFE = 95 %, T_meshes = 89.770509 %, lambda_LXe = 100 cm); 
## To be iteratively determined from data later
fit_parameters_eta_v1 = np.array([[ 0.76784314,  0.04281434,  0.00264452, -0.16502946],
       [ 0.86618744, -0.04220011,  0.00349323, -0.21111826],
       [ 0.76836278,  0.0427951 ,  0.00265082, -0.16546435],
       [ 0.76766095, -0.04280319,  0.00264167, -0.16492769],
       [ 0.86537262, -0.04224966,  0.00347602, -0.20996557],
       [ 0.76728103,  0.04285181,  0.00263252, -0.16423993],
       [-0.02781789,  1.27335415, -0.01554958,  0.56522927]])
fit_parameters_eta_v2 = np.array([[  0.5862075 ,  30.03753042,  -0.34682365,   4.18897093,  2.59854272],
       [  0.6389123 ,  29.31198247,  -0.3388027 ,   4.04470674,  2.63577199],
       [  0.58620791,  29.9885029 ,  -0.34004891,   4.17451146,  2.60269506],
       [  0.58610133,  30.01226409,  -0.33813655,   4.17468471,  2.59997022],
       [  0.6394532 ,  29.50418414,  -0.37196722,   4.10965579,  2.62267883],
       [  0.58654552,  30.13917691,  -0.36157398,   4.22025887,  2.59104887],
       [  0.53325685,  39.60254022, -18.11911918,  20.76666722,  2.24994645]])
fit_parameters_eta_v3 = np.array([[14.98906971,  6.01402517,  0.6260277 ,  0.03175688],
       [15.0328818 ,  5.88840721,  0.67446688,  0.03821948],
       [14.99645946,  6.01064072,  0.62595618,  0.03176641],
       [14.99212962,  6.01742089,  0.62604177,  0.03172401],
       [15.00980035,  5.90048797,  0.67550156,  0.03817195],
       [14.96863869,  6.02563842,  0.62689395,  0.03171014],
       [14.42475061,  4.6118038 ,  0.46425292,  0.09923257]])
fit_parameters_eta_v4 = np.array([[ 1.50458218e+01,  5.24536009e+00,  5.56365828e-01,  7.80021868e-02, -8.70727228e-04],
       [ 1.50495274e+01,  5.12846441e+00,  5.96407767e-01,  9.21654167e-02, -1.05651169e-03],
       [ 1.50530290e+01,  5.24107977e+00,  5.56230036e-01,  7.80704244e-02, -8.71867254e-04],
       [ 1.50492943e+01,  5.24533429e+00,  5.56099760e-01,  7.81532305e-02, -8.74082128e-04],
       [ 1.50294150e+01,  5.14635567e+00,  5.97943157e-01,  9.16457735e-02, -1.04685791e-03],
       [ 1.50284661e+01,  5.25817388e+00,  5.57305438e-01,  7.77962504e-02, -8.67398231e-04],
       [ 1.45326430e+01,  5.59447419e+00,  6.41420473e-01,  -5.34046071e-02,  4.27492085e-03]])

## Reconstruction Algorithm

In [5]:
#### Definition and selection of radial LRF model

## Position Reconstruction in LUX, https://arxiv.org/abs/1710.02752v2
def eta_v1(r, A, gamma, m, b):
    return A / (1 + gamma ** 2 * r ** 2) ** (3/2) + m * r + b

## Position Reconstruction in a Dual Phase Xenon Scintillation Detector, https://arxiv.org/abs/1112.1481
def eta_v2(r, A, r0, a, b, alpha):
    return A * exp( - a * (r/r0) / (1 + (r/r0) ** (1 - alpha)) - b / (1 + (r/r0) ** (- alpha)))   

## Modified Woods Saxon Potential
def eta_v3(r, R, a, V0, d):
    return V0 / (1 + np.exp((r - R) / a)) + d

## Modified Woods Saxon Potential + Linear
def eta_v4(r, R, a, V0, d, m):
    return V0 / (1 + np.exp((r - R) / a)) + d + m * r

## Choice for LRF model
model = eta_v2                  # default:  eta_v2
modelstring = 'eta_v2'          # default: 'eta_v2'

In [6]:
#### Definition of metric q to be minimized

## Function q to minimize for position reconstruction 
## Choose from: 'ChiSquared' 'LogLikelihoodRatio' 'WeightedChiSquared'
minimizer = 'ChiSquared'        # default: 'ChiSquared'

## Normalize LRFs to sum LRFs for all PMTs
normalization_to_sum = True     # default: True

## Ignore PMTs (i.e. set HF and LRF to 0) with light share below a specific threshold
cut_LS_PMT = False              # default: False

if cut_LS_PMT == True:
    cut_LS_PMT_threshold = 0.04 # start playing around with 0.04 - 0.05

## Mask to exclude PMTs not passing cut_LS_PMT_threshold
def pass_cut_LS_PMT_threshold(HFs_input):
    if cut_LS_PMT == True:
        return (HFs_input >= cut_LS_PMT_threshold).astype(int)
    if cut_LS_PMT == False:
        return (np.array([1]*7))

## LRF values for above conditions

if normalization_to_sum == False: ## radial LRFs only
    modelstring_save = modelstring
    def LRF_PMTi(x, y, PMT, fitparameters):
        fp = fitparameters[PMT-1]
        return model(np.sqrt((x - PMT_positions['PMT_position_x'][PMT])**2 + (y - PMT_positions['PMT_position_y'][PMT])**2), *fp)
    def LRF_PMTs(x, y, HFs_input, fitparameters):
        return np.array([LRF_PMTi(x, y, j, fitparameters) for j in range(1,8)])*pass_cut_LS_PMT_threshold(HFs_input)
    
if normalization_to_sum == True: ## radial LRFs normalized to sum radial LRFs
    modelstring_save = modelstring+'_overSum'
    def LRF_PMTi(x, y, PMT, fitparameters):
        fp = fitparameters[PMT-1]
        return model(np.sqrt((x - PMT_positions['PMT_position_x'][PMT])**2 + (y - PMT_positions['PMT_position_y'][PMT])**2), *fp)
    def LRF_PMTs(x, y, HFs_input, fitparameters):
        LRF_PMTs_array = np.array([LRF_PMTi(x, y, j, fitparameters) for j in range(1,8)])*pass_cut_LS_PMT_threshold(HFs_input)
        LRF_PMTs_array_uncut = np.array([LRF_PMTi(x, y, j, fitparameters) for j in range(1,8)])
        return LRF_PMTs_array / (np.sum(LRF_PMTs_array_uncut))

## Unweighted Chi squared from LRFs:

if minimizer == 'ChiSquared':
    modelstring_save = modelstring_save+'_Chi2'
    
    ## numerator [top of the fraction]
    def numerator(x, y, HFs_input, fitparameters):
        return (LRF_PMTs(x, y, HFs_input, fitparameters) - HFs_input*pass_cut_LS_PMT_threshold(HFs_input))**2
    
    ## denominator [bottom of a fraction]
    def denominator(x, y, fitparameters):
        return LRF_PMTs(x, y, (np.array([1]*7)), fitparameters)
    
    ## Chi²
    def q(x, y, HFs_input, fitparameters):
        return np.sum(numerator(x, y, HFs_input, fitparameters) / denominator(x, y, fitparameters))

## Weighted Chi squared from LRFs:
    
if minimizer == 'WeightedChiSquared':
    modelstring_save = modelstring_save+'_WChi2'
    sigma = 1./3. # excess noise factor for each PMT, default: 1/3
    
    ## numerator [top of the fraction]
    def numerator(x, y, HFs_input, fitparameters):
        return (LRF_PMTs(x, y, HFs_input, fitparameters) - HFs_input*pass_cut_LS_PMT_threshold(HFs_input))**2
    
    ## denominator [bottom of a fraction]
    def denominator(x, y, fitparameters):
        return LRF_PMTs(x, y, (np.array([1]*7)), fitparameters) * (1 + sigma**2)
    
    ## Chi²
    def q(x, y, HFs_input, fitparameters):
        return np.sum(numerator(x, y, HFs_input, fitparameters) / denominator(x, y, fitparameters))

## Log Likelihood Ratio from LRFs:    

if minimizer == 'LogLikelihoodRatio':
    modelstring_save = modelstring_save+'_LLR'
    
    def q(x, y, HFs_input, fitparameters):
        summed = 0
        for i in range(0,7):
            if pass_cut_LS_PMT_threshold(HFs_input)[i] > 0:
                summed += 2 * (LRF_PMTs(x, y, HFs_input, fitparameters)[i] - HFs_input[i]*pass_cut_LS_PMT_threshold(HFs_input)[i] + HFs_input[i]*pass_cut_LS_PMT_threshold(HFs_input)[i] * np.log( (HFs_input[i]*pass_cut_LS_PMT_threshold(HFs_input)[i]) / (LRF_PMTs(x, y, HFs_input, fitparameters)[i]) ))
        return summed

In [7]:
#### Reconstruct position inside volume

## Function to return non-negative value corresponding to (radial position - radius TPC) if inside TPC,
## used for constraints

def insidevolume(inputs):
    return (p_TPC_radius - np.sqrt(inputs[0]**2 + inputs[1]**2))

## Reconstruct position by minimizing q

def reconstructed(HFs_input, fitparameters):
    reconstruct = lambda x: q(x[0], x[1], HFs_input, fitparameters) # The objective function to be minimized.
    x0 = [0.001,0.001] # Initial guess.
    bnds = ((-35, 35), (-35, 35)) # Bounds on variables for L-BFGS-B, TNC, SLSQP and trust-constr methods.
    meth = 'SLSQP' # Type of solver. 'BFGS' / 'L-BFGS-B' / 'SLSQP' / ...
    cons = ({'type': 'ineq', "fun": insidevolume}) # Constraints definition (only for COBYLA, SLSQP and trust-constr).
    ################################
    # The optimization result represented as a OptimizeResult object:
    #res = minimize(reconstruct, x0, method = meth)
    #res = minimize(reconstruct, x0, bounds = bnds)
    res = minimize(reconstruct, x0, method=meth, constraints=cons)
    return res

## Return reconstructed position as array([x, y])
## using fitparameters from above
## input: np.array of lenght (number top PMTs) i.e. 7 in XeBRA with pulse area fractions S2 signals
##        or corresponding hits fraction for PMTs in numbered order
##        and
##        fitparameters of the LRFs

def reconstructed_position(input_array, fitp):
    HFs_inp = input_array / np.sum(input_array) # normalize to one to correspond to light share
    return reconstructed(HFs_inp, fitp).x

## Iterative Reconstruction

In [8]:
#### General settings

## Maximum number of iterations
iterations = 5

## Fiducial radial cuts in mm
rfiducial = 20          # fiducial cut on radial position in TPC, default: 20
rfiducial_under = 15    # fiducial cut on radial distance from center individual PMT, default: 15

## Fiducial radial cut central PMT in mm 
rfiducial_central = 32  # fiducial cut on radial position in TPC, default: 34 / 30

## Fiducial radial cut 0th iteration in mm
rfiducial0 = 34         # default: 34
rfiducial0_under = -1   # default: -1, set to -1 to make inactive
rfiducial0_central = 34 # default: 34

In [9]:
HFs_measured = np.array([0.26336634, 0.44257426, 0.05346535, 0.02673267, 0.02376238, 0.04257426, 0.14752475])
exec('fitparameters = fit_parameters_'+modelstring) # fit parameters for selcted model

reconstructed_position(HFs_measured, fitparameters)

array([-30.14597879, -15.46811501])